In [25]:
%pip install selenium
%pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import time
from selenium.webdriver.common.keys import Keys

In [2]:
# Credenciales de Twitter
USERNAME = 'ScrapeoUPM'
PASSWORD = 'scrapeafutbol'
EMAIL = 'oscar.marin@alumnos.upm.es'

SCROLL_PAUSE_TIME = 3  # Tiempo de espera para que carguen nuevos tweets

In [8]:
def login(driver, username, email, password):
    
    # Navegar a la página de inicio de sesión de Twitter
    driver.get('https://twitter.com/login')
    time.sleep(7)  # Esperar a que cargue la página

    # Buscar el campo del nombre de usuario e ingresar el nombre de usuario o correo
    username_input = driver.find_element(By.NAME, 'text')
    username_input.send_keys(username)
    username_input.send_keys(Keys.RETURN) #Presionar tecla enter
    time.sleep(2)
    
    try:
        # Buscar el campo de confirmación de correo
        email_input = driver.find_element(By.NAME, 'text')
        email_input.send_keys(email)
        email_input.send_keys(Keys.RETURN)
        time.sleep(2)
    except:
        pass
    
    # Buscar el campo de contraseña e ingresar la contraseña
    password_input = driver.find_element(By.NAME, 'password')
    password_input.send_keys(password)
    password_input.send_keys(Keys.RETURN)
    time.sleep(5)

def scrap_tweets(driver, n_tweets: int, word_search: str, date_range: tuple):
    
    login(driver, USERNAME, EMAIL, PASSWORD)
    
    word_search = word_search.replace(' ', '%20')
    date_range = f'%20until%3A{date_range[1]}%20since%3A{date_range[0]}'

    # Navegar a la página de búsqueda de Twitter
    driver.get(f'https://twitter.com/search?q={word_search}{date_range}')
    time.sleep(5)  # Esperar a que cargue la página

    n = 0
    tweets = []
    while n < n_tweets:
        # Extraer los tweets visibles en este momento
        new_tweets = driver.find_elements(By.CSS_SELECTOR, 'article[data-testid="tweet"]')
        
        # Imprimir el texto de los tweets extraídos
        for tweet in new_tweets:
            tweet_text = tweet.find_element(By.CSS_SELECTOR, 'div[data-testid="tweetText"]')
            tweet_date = tweet.find_element(By.CSS_SELECTOR, 'time').get_attribute('datetime')
            tweets.append({'text': tweet_text.text, 'date': tweet_date})
            n += 1
            if n == n_tweets:
                break

        # Hacer scroll hasta el final de la página
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)

    return tweets

In [9]:
q = 'Ferran Torres'
since = '2023-09-01'
until = '2024-06-01'
n_tweets = 100

chrome_options = Options()
chrome_options.add_argument("--headless")  # Ejecuta Chrome en modo headless

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
tweets = scrap_tweets(driver, n_tweets, q, (since, until))

# Cerrar el navegador
driver.quit()
tweets